In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

data = Path("../data")
plot_path = data / "plots"

map_path = data / "graph3_map"
geojson_path = map_path / "geojson"
stat_path = map_path / "stats"
country_stat_path = stat_path / "countries"
studio_stat_path = stat_path / "studios"

# Animes

In [2]:
AnimeList = pd.read_csv(data / "AnimeList.csv")
unique_anime_list = AnimeList["title"].unique()
unique_anime_list = sorted(unique_anime_list)
print(len(unique_anime_list))

14477


In [3]:
anime_cleaned = pd.read_csv(data / "anime_cleaned.csv")
unique_anime_cleaned = anime_cleaned["title"].unique()
unique_anime_cleaned = sorted(unique_anime_cleaned)
print(len(unique_anime_cleaned))

6668


In [4]:
anime_list_not_cleaned = set(unique_anime_list) - set(unique_anime_cleaned)
len(anime_list_not_cleaned)

7809

In [5]:
anime_cleaned_not_list = set(unique_anime_cleaned) - set(unique_anime_list)
len(anime_cleaned_not_list)

0

# Character-Animes

In [6]:
characters = pd.read_csv(data / "Anime_Traits.csv")
num_unique_characters = characters["Id"].nunique()
print("Number of unique characters: ", num_unique_characters)
characters

Number of unique characters:  119824


,Id,Names,Hair_Color,Gender,Tags,Anime,Manga
0,0,L,Black Hair,Male,"['Analytical', 'Barefoot', 'Detective', 'Eye B...","['Death Note', 'Death Note Rewrite 1: Visions ...","['Death Note', 'Death Note: L Change the World..."
1,1,Kakashi HATAKE,Grey Hair,Male,"['Adult', 'Anti-Gravity Hair', 'Beastmaster', ...","['Boruto: Naruto Next Generations', 'Boruto: N...",['Boruto: Naruto the Movie Tokubetsu Bangai-he...
2,2,Edward ELRIC,Blonde Hair,Male,"['Alchemist', 'Arm Blades', 'Artificial Limb',...","['Fullmetal Alchemist', 'Fullmetal Alchemist: ...","['Fullmetal Alchemist', 'Fullmetal Alchemist G..."
3,3,Levi,Black Hair,Male,"['Adult', 'Military', 'Ruthless', 'Stoic', 'Sw...","['Attack on Skytree', 'Attack on Titan', 'Atta...","['Attack on Titan', 'Attack on Titan: Junior H..."
4,4,Light YAGAMI,Brown Hair,Male,"['Analytical', 'Big Ego', 'Charismatic', 'God ...","['Death Note', 'Death Note Rewrite 1: Visions ...",['Death Note']
...,...,...,...,...,...,...,...
119820,123369,Narrator,NaN,Male,['Gold Lightan: Golden Warrior'],NaN,NaN
119821,123370,Aoi,Purple Hair,Female,"['Editor', 'Hair Antenna']",['Kemeko Deluxe!'],['Kemeko DX!']
119822,123371,Kiriko,Black Hair,Female,"['Eye Patch', 'Robot', 'Rosy Cheeks']",['Kemeko Deluxe!'],['Kemeko DX!']
119823,123372,Ryouta MINAMINO,Blonde Hair,Male,"['Anti-Gravity Hair', 'Glasses', 'Headband']",['Kemeko Deluxe!'],['Kemeko DX!']


In [7]:
import re

# Add a column "Anime1" that is the copy of the column "Anime"
characters["Anime1"] = characters["Anime"]

characters_not_null_anime = characters[characters["Anime1"].notnull()]

characters_not_null_anime.loc[:, "Anime1"] = (
    characters_not_null_anime.loc[:, "Anime1"]
    .str.replace(r"^\[", "")
    .str.replace(r"\]$", "")
)

characters_not_null_anime.loc[:, "Anime1"] = characters_not_null_anime.loc[
    :, "Anime1"
].str.split(",")
characters_not_null_anime = characters_not_null_anime.explode("Anime1")

# strip every string
characters_not_null_anime.loc[:, "Anime1"] = characters_not_null_anime.loc[
    :, "Anime1"
].str.strip()


# For every row where Anime1 starts with a quote and/or ends with a quote, remove the quote(s)
characters_not_null_anime.loc[:, "Anime1"] = (
    characters_not_null_anime.loc[:, "Anime1"]
    .str.replace(r"^\"", "")
    .str.replace(r"\"$", "")
    .str.replace(r"^\'", "")
    .str.replace(r"\'$", "")
    .str.strip()
)

# For every row of to_handle whose Anime1 string contains a substring that starts with "[email" and ends with "protected]", remove the substring
characters_not_null_anime.loc[:, "Anime1"] = (
    characters_not_null_anime.loc[:, "Anime1"]
    .apply(lambda x: re.sub(r" \[email.*protected\]", "", x).strip())
)
# Remove the row with Anime1 equal to "[email\x0aprotected]" whose Anime1 string contains a substring that starts with "[email" and ends with "protected]", remove the substring
characters_not_null_anime = characters_not_null_anime[
    ~characters_not_null_anime["Anime1"].str.contains(r"\[email.*protected\]")
]

# merge the results with the original dataframe
characters = pd.concat(
    [
        characters_not_null_anime,
        characters[characters["Anime"].isnull()],
    ],
    axis=0,
)
# assert characters["Id"].nunique() == num_unique_characters-1
characters

/tmp/ipykernel_35618/2592428104.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r"^\[", "")
/tmp/ipykernel_35618/2592428104.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r"\]$", "")
/tmp/ipykernel_35618/2592428104.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  characters_not_null_anime.loc[:, "Anime1"] = (
/tmp/ipykernel_35618/2592428104.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

,Id,Names,Hair_Color,Gender,Tags,Anime,Manga,Anime1
0,0,L,Black Hair,Male,"['Analytical', 'Barefoot', 'Detective', 'Eye B...","['Death Note', 'Death Note Rewrite 1: Visions ...","['Death Note', 'Death Note: L Change the World...",Death Note
0,0,L,Black Hair,Male,"['Analytical', 'Barefoot', 'Detective', 'Eye B...","['Death Note', 'Death Note Rewrite 1: Visions ...","['Death Note', 'Death Note: L Change the World...",Death Note Rewrite 1: Visions of a God
0,0,L,Black Hair,Male,"['Analytical', 'Barefoot', 'Detective', 'Eye B...","['Death Note', 'Death Note Rewrite 1: Visions ...","['Death Note', 'Death Note: L Change the World...",Death Note Rewrite 2: L's Successors
1,1,Kakashi HATAKE,Grey Hair,Male,"['Adult', 'Anti-Gravity Hair', 'Beastmaster', ...","['Boruto: Naruto Next Generations', 'Boruto: N...",['Boruto: Naruto the Movie Tokubetsu Bangai-he...,Boruto: Naruto Next Generations
1,1,Kakashi HATAKE,Grey Hair,Male,"['Adult', 'Anti-Gravity Hair', 'Beastmaster', ...","['Boruto: Naruto Next Generations', 'Boruto: N...",['Boruto: Naruto the Movie Tokubetsu Bangai-he...,Boruto: Naruto the Movie
...,...,...,...,...,...,...,...,...
119813,123357,Kitajima,Black Hair,Male,['A Town Where You Live (2014)'],NaN,NaN,NaN
119814,123358,Nakashiki,Brown Hair,Female,['A Town Where You Live (2014)'],NaN,NaN,NaN
119818,123367,Narrator,NaN,Male,['Gasshin Sentai Mechander Robo'],NaN,NaN,NaN
119820,123369,Narrator,NaN,Male,['Gold Lightan: Golden Warrior'],NaN,NaN,NaN


In [8]:
unique_anime_characters = characters["Anime1"].unique()
unique_anime_characters = unique_anime_characters[~pd.isnull(unique_anime_characters)]
unique_anime_characters = sorted(unique_anime_characters)
print(len(unique_anime_characters))

17388


In [9]:
print(len(unique_anime_list))

14477


# Compare

In [10]:
# Animes that are in AnimeList but not in Anime_Traits
anime_list_not_characters = set(unique_anime_list) - set(unique_anime_characters)
anime_list_not_characters = sorted(anime_list_not_characters)
print(len(anime_list_not_characters))

11432


In [11]:
# Animes that are in Anime_Traits but not in AnimeList
anime_characters_not_list = set(unique_anime_characters) - set(unique_anime_list)
anime_characters_not_list = sorted(anime_characters_not_list)
print(len(anime_characters_not_list))

14343


In [12]:
# Animes that are in Anime_List and Anime_Traits
anime_list_and_characters = set(unique_anime_list) & set(unique_anime_characters)
anime_list_and_characters = sorted(anime_list_and_characters)
print(len(anime_list_and_characters))

3045


In [13]:
print(*anime_characters_not_list, sep="\n")


#000000 - Ultra Black
#Compass
#Killstagram
$10
$100 is Too Cheap
&
& Other Delusions! Rikka Version
& Other Delusions! Rikka Version Lite: My Brother 2
(G) Edition
(Sokuhou) Class no Ikemen ga Watashi no Shinkyoku ni Muchuu na Ken.
*Luna: classroom
+99 Reinforced Wooden Stick
+A-Channel
+Anima
+a no Tachiichi
...Junai no Seinen
.hack // The Movie
.hack//4koma
.hack//AI Buster (Light Novel)
.hack//Another Birth (Light Novel)
.hack//G.U. (Light Novel)
.hack//G.U. Begins
.hack//G.U. Trilogy Parody
.hack//G.U.+
.hack//GIFT
.hack//GnU
.hack//LIMINALITY
.hack//Legend Of The Twilight
.hack//Legend of the Twilight
.hack//Link
.hack//Quantum: Go
.hack//SIGN
.hack//SIGN - Intermezzo
.hack//SIGN - Unison
.hack//XXXX
.hack//ZERO (Light Novel)
0 kara Hajimeru Manga Kyoushitsu
000 Light Years - IGZA
000 Years
000 Years Gets Unfairly Villainized
009 Re:Cyborg x NTV ZIP!
009 Re:Cyborg: Seishain Cyborg 003
01 Monkey
07-Ghost: Children
090: Eko to Issho.
1 Love 9
1 Plus 1 (Soru)
1 RT de Nakawarui Nonk